# Estimator 
### Last Modified: August 24, 2020

To run, press SHIFT+ENTER on each cell. <br>
To get to next cell, press the down arrow after clicking on a blank spot of the cell. <br>
<b> Likewise, you can click on "Restart & Run All" under the Kernel menu (may need to re-run cells with dropdown boxes)</b><br>
Cells can be toggled on and off by running the cell preceding this message.
## If asked to "Try and re-run cell", then click on a blank spot of the cell until no dropdown box or button is selected and press SHIFT+ENTER

In [2]:
# Widgets
from IPython.display import display, clear_output, HTML, FileLink
import ipywidgets as widgets
from ipywidgets import Layout

# Imports
import pandas as pd
import numpy as np
import math

import io
import requests

import time
from datetime import date

import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Variables
scope = "https://spreadsheets.google.com/feeds"
credentials = ServiceAccountCredentials.from_json_keyfile_name("matcher-272116-8801bce55bcb.json", scope)
gs = gspread.authorize(credentials)

estimator_sheet_url = "https://docs.google.com/spreadsheets/d/1EhbS_vM28BQWW2SVCS1RefWD9tzDrWbYT2MdSDlobVM"

# Print Matcher Sheet URL
print("Go to {0} to view Estimator Sheet".format(estimator_sheet_url))

Go to https://docs.google.com/spreadsheets/d/1EhbS_vM28BQWW2SVCS1RefWD9tzDrWbYT2MdSDlobVM to view Estimator Sheet


In [3]:
# Allows window height to be larger
# Doesn't seem to work in Binder?
HTML('''
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:50000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>''');

In [4]:
# Toggle Code Button
HTML('''<script>
code_show=true;
function code_toggle() {
    if (code_show){
    $('div.input').hide();
    } else {
    $('div.input').show();
    }
    code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the code."></form>
''')

# Journey Data

In [11]:
# Creates current journeys update button and corresponding output
update_current_journeys_button = widgets.Button(description = "Update Current Journeys", layout = Layout(width = "200px"))
out_update_current_journeys = widgets.Output()

def update_current_journeys(_) :
    with out_update_current_journeys :
        # Clears printed output
        clear_output()
        
        # Updates current journeys
        print("Updating Current_Journeys", end = "\r")
        
        # Tells you how long it took to run
        # Initializes start time
        start_time = time.time()
        
        # Opens and clears current journeys sheet
        current_journeys_sheet = gs.open_by_url(estimator_sheet_url).worksheet("Current_Journeys")
        current_journeys_sheet.clear()
        
        # Updates Journey worksheet with cyfe board
        current_journeys_sheet.update("A1", gs.open_by_url(estimator_sheet_url).worksheet("URLs").get("B1")[0][0][1:], raw = False)
        
        # Waits for cell A1 to be the value "journey_id"
        while (current_journeys_sheet.acell("A1").value != "#N/A" and current_journeys_sheet.acell("A1").value != "job_id") :
            time.sleep(5)
            
        # Copies cells to save journey
        current_journeys_sheet.update("A1:R", current_journeys_sheet.get("A1:R"), raw = False)
        
        # Prints out how long it took to run
        print("Updated Current_Journeys Worksheet in {0:.3f} secs to run".format(time.time() - start_time))
        # Prints if cyfe board was empty
        if current_journeys_sheet.acell("A1").value == "#N/A" :
            print("No journeys loaded")
            
        # Clears results in Information
        gs.open_by_url(estimator_sheet_url).values_clear("Information!M4:R")
            
# Sets current journeys update button with function
update_current_journeys_button.on_click(update_current_journeys)

# Displays widgets
widgets.VBox([update_current_journeys_button, out_update_current_journeys])

# Estimate Number of Views and Expressions